# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,iskateley,67.6803,53.1512,-13.32,97,100,3.58,RU,1699909976
1,1,port saint john's,-31.6229,29.5448,17.08,81,100,4.12,ZA,1699909976
2,2,ta`u,-14.2336,-169.5144,27.82,83,89,3.88,AS,1699909976
3,3,timbuktu,16.7735,-3.0074,28.86,9,0,4.70,ML,1699909976
4,4,alexandria,31.2156,29.9553,22.45,64,0,2.96,EG,1699909701


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", 
    size = "Humidity"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [53]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_ideal_weather_df = city_data_df[(city_data_df["Max Temp"] >= 22) & (city_data_df["Max Temp"] <= 25)]

# Drop any rows with null values
city_data_ideal_weather_df = city_data_ideal_weather_df.dropna(how="any")

# Display sample data
city_data_ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,alexandria,31.2156,29.9553,22.45,64,0,2.96,EG,1699909701
23,23,adamstown,-25.0660,-130.1015,23.02,86,91,8.74,PN,1699909978
32,32,alice springs,-23.7000,133.8833,24.75,31,16,3.60,AU,1699909979
54,54,castro-urdiales,43.3829,-3.2204,22.12,66,100,1.40,ES,1699909982
55,55,vicente guerrero,19.1194,-98.1708,24.66,48,61,0.40,MX,1699909983
...,...,...,...,...,...,...,...,...,...,...
538,538,ouesso,1.6136,16.0517,22.28,98,29,0.45,CG,1699910068
544,544,pointe aux piments,-20.0628,57.5131,24.18,72,2,2.71,MU,1699910069
554,554,sambava,-14.2667,50.1667,23.46,85,71,2.02,MG,1699910071
565,565,sandakan,5.8402,118.1179,24.52,94,58,1.57,MY,1699910072


### Step 3: Create a new DataFrame called `hotel_df`.

In [19]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_ideal_weather_df.loc[:, ["City", "Country", "Lat", "Lng", "Humidity"]].copy()
hotel_df

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
4,alexandria,EG,31.2156,29.9553,64,
23,adamstown,PN,-25.0660,-130.1015,86,
32,alice springs,AU,-23.7000,133.8833,31,
54,castro-urdiales,ES,43.3829,-3.2204,66,
55,vicente guerrero,MX,19.1194,-98.1708,48,
...,...,...,...,...,...,...
538,ouesso,CG,1.6136,16.0517,98,
544,pointe aux piments,MU,-20.0628,57.5131,72,
554,sambava,MG,-14.2667,50.1667,85,
565,sandakan,MY,5.8402,118.1179,94,


In [20]:
# Check Geoapify for Hotels
target_city = "Adamstown, Pitcairn Islands"

# Build the endpoint URL
target_url = f"https://api.geoapify.com/v1/geocode/search?text={target_city}&format=json&apiKey={geoapify_key}"

# Run a request to endpoint and convert result to json
geo_data = requests.get(target_url).json()

# Print the json (pretty printed)
print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "query": {
        "parsed": {
            "city": "adamstown",
            "country": "pitcairn islands",
            "expected_type": "city"
        },
        "text": "Adamstown, Pitcairn Islands"
    },
    "results": [
        {
            "address_line1": "Adamstown",
            "address_line2": "Pitcairn Islands",
            "bbox": {
                "lat1": -25.086667,
                "lat2": -25.046667,
                "lon1": -130.120205,
                "lon2": -130.080205
            },
            "category": "populated_place",
            "city": "Adamstown",
            "country": "Pitcairn Islands",
            "country_code": "pn",
            "datasource": {
                "attribution": "\u00a9 OpenStreetMap contributors",
                "license": "Open Database License",
                "sourcename": "openstreetmap",
                "url": "https://www.openstreetmap.org/copyright"
            },
            "formatted": "Adamstown, Pitcairn Islands",
   

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Print the API response for debugging
    #print(name_address.text)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alexandria - nearest hotel: Smoha Zahran Haus (7th floor)
adamstown - nearest hotel: No hotel found
alice springs - nearest hotel: Aurora Alice Springs
castro-urdiales - nearest hotel: Hotel La Ronda
vicente guerrero - nearest hotel: HOTEL SNAYLF
avarua - nearest hotel: Paradise Inn
flying fish cove - nearest hotel: Christmas Island Lodge
dompu - nearest hotel: No hotel found
namibe - nearest hotel: Hotel Chik Chik Namibe
lompoc - nearest hotel: Embassy Suites by Hilton Lompoc Central Coast
karbala - nearest hotel: فندق القمر
tambura - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
el alto - nearest hotel: No hotel found
coatepec harinas - nearest hotel: No hotel found
nuku'alofa - nearest hotel: City Hotel
subaykhan - nearest hotel: No hotel found
non sang - nearest hotel: No hotel found
saint-paul - nearest hotel: Hôtel d'application
duong dong - nearest hotel: Kim Hông Ngoc
ayacucho - nearest hotel: Hotel Las Americas
bardai - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
4,alexandria,EG,31.2156,29.9553,64,Smoha Zahran Haus (7th floor)
23,adamstown,PN,-25.0660,-130.1015,86,No hotel found
32,alice springs,AU,-23.7000,133.8833,31,Aurora Alice Springs
54,castro-urdiales,ES,43.3829,-3.2204,66,Hotel La Ronda
55,vicente guerrero,MX,19.1194,-98.1708,48,HOTEL SNAYLF
...,...,...,...,...,...,...
538,ouesso,CG,1.6136,16.0517,98,Auberge Nianina
544,pointe aux piments,MU,-20.0628,57.5131,72,Hotel Villas Mon Plaisir
554,sambava,MG,-14.2667,50.1667,85,Royal Sava Hôtel
565,sandakan,MY,5.8402,118.1179,94,Hotel City Star


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [57]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", 
    size = "Humidity",
    hover_cols=["Country", "Hotel Name"]
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,Country,Hotel Name)